# MLflow Interface

## Table of Contents

1. [Set Up MLflow Credential and Environment values](#Set-Up-MLflow-Credential-and-Environment-values)
2. [Get Registerd Model List and Model Details](#Get-Registerd-Model-List-and-Model-Details)

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
import requests
import ipywidgets as widgets

from IPython.display import display

In [ ]:
domain = "ezua1.local"
mlflow_username = "chowoocheol@gmail.com"
mlflow_password = "userdemo1!"

# Set Up MLflow Credential and Environment values

Set up MLflow for model tracking and register the model. Include authentication if necessary.

In [ ]:
# Add heading
heading = widgets.HTML("<h2>MLflow Credentials</h2>")
display(heading)

domain_input = widgets.Text(description='Domain:', placeholder="ua.ezm.host")
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

domain = None
mlflow_username = None
mlflow_password = None

def submit_button_clicked(b):
    global domain, mlflow_username, mlflow_password
    domain = domain_input.value
    mlflow_username = username_input.value
    mlflow_password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(domain_input, username_input, password_input, submit_button, success_message)

In [ ]:
token_url = f"https://keycloak.{domain}/realms/UA/protocol/openid-connect/token"

data = {
    "username" : mlflow_username,
    "password" : mlflow_password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
os.environ['MLFLOW_TRACKING_TOKEN'] = token
os.environ["AWS_ACCESS_KEY_ID"] = os.environ['MLFLOW_TRACKING_TOKEN']
os.environ["AWS_SECRET_ACCESS_KEY"] = "s3"
os.environ["AWS_ENDPOINT_URL"] = 'http://local-s3-service.ezdata-system.svc.cluster.local:30000'
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.environ["AWS_ENDPOINT_URL"]
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_TRACKING_INSECURE_TLS"] = "true"
os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow.mlflow.svc.cluster.local:5000"

# Get Registerd Model List and Model Details

Set up MLflow client and get information of registered models

In [ ]:
# Create an instance of the MLflow client
client = MlflowClient()

In [ ]:
# Function to get list of registered models
def list_registered_models():
    models = client.search_registered_models()
    return models

# Function to get details of a specific model
def get_model_details(model_name):
    try:
        # 모델의 모든 버전 정보를 검색
        versions = client.search_model_versions(f"name='{model_name}'")
        model_details = []
        
        # 각 버전의 상세 정보 추출
        for version in versions:
            detail = {
                "version": version.version,
                "run_id": version.run_id,
                "source": version.source,
                "status": version.status,
                "created_time": version.creation_timestamp,
                "last_updated_time": version.last_updated_timestamp
            }
            model_details.append(detail)
        
        return model_details
    
    except Exception as e:
        print(f"Error retrieving details for model '{model_name}': {e}")
        return None

In [ ]:
# 등록된 모델 리스트 가져오기
registered_models = list_registered_models()
print("Registered Models:")
for model in registered_models:
    print(f"- {model.name}")

# Prompt the user to enter the model name they want to query
model_name = input("Enter the registered model name you want to query: ")

# Retrieve and display the details for the specified model
details = get_model_details(model_name)

if details:
    print(f"\nDetails for model '{model_name}':")
    for detail in details:
        print(detail)
else:
    print(f"No details found for model '{model_name}'")